# Agent-to-Agent (A2A) Communication with Google ADK

## Introduction

**Agent-to-Agent (A2A)** communication allows multiple AI agents to:
- Collaborate on complex tasks
- Share information and context
- Delegate work to specialists
- Coordinate multi-step workflows

Google ADK provides **built-in A2A protocol** for seamless agent communication.

### What You'll Learn
- A2A protocol fundamentals
- Message passing between agents
- Multi-agent coordination patterns
- Building agent teams
- Production A2A systems

## Part 1: A2A Protocol Basics

### How A2A Works

```
┌──────────────┐       A2A Message        ┌──────────────┐
│   Agent A    │ ─────────────────────►   │   Agent B    │
│ (Coordinator)│                          │ (Specialist) │
│              │ ◄─────────────────────   │              │
└──────────────┘      Response            └──────────────┘

A2A Message Format:
{
  "from_agent": "agent_a_id",
  "to_agent": "agent_b_id",
  "message_type": "request",
  "payload": {
    "task": "analyze_data",
    "data": {...}
  },
  "context": {...},
  "metadata": {...}
}
```

## Part 2: Simple A2A Example

In [ ]:
from google.cloud.adk import Agent, A2AMessage, A2AProtocol
from typing import Dict, Any

# Agent 1: Research Agent
class ResearchAgent(Agent):
    """Agent that researches topics"""
    
    def __init__(self):
        super().__init__(
            name="researcher",
            description="I research topics and gather information",
            a2a_enabled=True  # Enable A2A communication
        )
    
    async def handle_a2a_request(self, message: A2AMessage) -> Dict[str, Any]:
        """Handle incoming A2A requests"""
        task = message.payload.get("task")
        topic = message.payload.get("topic")
        
        if task == "research":
            # Perform research
            research_results = {
                "topic": topic,
                "findings": [
                    f"Finding 1 about {topic}",
                    f"Finding 2 about {topic}",
                    f"Finding 3 about {topic}"
                ],
                "confidence": 0.85
            }
            
            return {
                "status": "success",
                "results": research_results
            }
        
        return {"status": "error", "message": "Unknown task"}

# Agent 2: Writer Agent
class WriterAgent(Agent):
    """Agent that writes content"""
    
    def __init__(self):
        super().__init__(
            name="writer",
            description="I write articles and reports",
            a2a_enabled=True
        )
    
    async def handle_a2a_request(self, message: A2AMessage) -> Dict[str, Any]:
        """Handle incoming A2A requests"""
        task = message.payload.get("task")
        data = message.payload.get("data")
        
        if task == "write_article":
            # Write article based on research
            article = f"""# Article about {data.get('topic')}
            
Based on research findings:
{chr(10).join('- ' + f for f in data.get('findings', []))}

Conclusion: This is an important topic worth exploring further.
"""
            
            return {
                "status": "success",
                "article": article
            }
        
        return {"status": "error", "message": "Unknown task"}

# Initialize agents
researcher = ResearchAgent()
writer = WriterAgent()

print(f"Agents initialized: {researcher.name}, {writer.name}")

## Part 3: Coordinator Agent Pattern

A coordinator agent delegates work to specialist agents.

In [ ]:
class CoordinatorAgent(Agent):
    """Coordinator that orchestrates other agents"""
    
    def __init__(self, team_agents: list):
        super().__init__(
            name="coordinator",
            description="I coordinate work between specialist agents",
            a2a_enabled=True
        )
        self.team = {agent.name: agent for agent in team_agents}
    
    async def run(self, user_input: str) -> str:
        """Process user request by coordinating agents"""
        print(f"Coordinator received: {user_input}")
        
        # Step 1: Send research request to researcher
        research_msg = A2AMessage(
            from_agent=self.agent_id,
            to_agent=self.team["researcher"].agent_id,
            message_type="request",
            payload={
                "task": "research",
                "topic": user_input
            }
        )
        
        print("→ Sending research request to researcher...")
        research_response = await self.send_a2a_message(research_msg)
        print(f"← Received research results")
        
        if research_response.get("status") != "success":
            return "Research failed"
        
        # Step 2: Send write request to writer
        write_msg = A2AMessage(
            from_agent=self.agent_id,
            to_agent=self.team["writer"].agent_id,
            message_type="request",
            payload={
                "task": "write_article",
                "data": research_response["results"]
            }
        )
        
        print("→ Sending write request to writer...")
        write_response = await self.send_a2a_message(write_msg)
        print(f"← Received article")
        
        if write_response.get("status") != "success":
            return "Writing failed"
        
        return write_response["article"]
    
    async def send_a2a_message(self, message: A2AMessage) -> Dict[str, Any]:
        """Send message to another agent"""
        target_agent = self.team.get(message.to_agent)
        if not target_agent:
            return {"status": "error", "message": "Agent not found"}
        
        return await target_agent.handle_a2a_request(message)

# Create coordinator with team
coordinator = CoordinatorAgent(team_agents=[researcher, writer])

# Test
async def test_coordinator():
    result = await coordinator.run("AI Agents")
    print("\nFinal Article:")
    print(result)

# await test_coordinator()
print("Coordinator agent created with team")

## Part 4: Multi-Agent Workflow Patterns

### Pattern 1: Pipeline (Sequential)

In [ ]:
class PipelineCoordinator(Agent):
    """Agent pipeline: A → B → C"""
    
    def __init__(self, pipeline_agents: list):
        super().__init__(
            name="pipeline_coordinator",
            description="Coordinates sequential agent pipeline",
            a2a_enabled=True
        )
        self.pipeline = pipeline_agents
    
    async def run(self, initial_input: Any) -> Any:
        """Run data through pipeline"""
        current_data = initial_input
        
        for agent in self.pipeline:
            print(f"→ Processing with {agent.name}...")
            
            message = A2AMessage(
                from_agent=self.agent_id,
                to_agent=agent.agent_id,
                message_type="process",
                payload={"data": current_data}
            )
            
            response = await agent.handle_a2a_request(message)
            current_data = response.get("output")
            
            print(f"← {agent.name} completed")
        
        return current_data

# Example agents for pipeline
class DataCollectorAgent(Agent):
    async def handle_a2a_request(self, msg):
        data = msg.payload["data"]
        return {"output": f"Collected: {data}"}

class DataProcessorAgent(Agent):
    async def handle_a2a_request(self, msg):
        data = msg.payload["data"]
        return {"output": f"Processed: {data}"}

class DataAnalyzerAgent(Agent):
    async def handle_a2a_request(self, msg):
        data = msg.payload["data"]
        return {"output": f"Analyzed: {data}"}

# Create pipeline
pipeline = PipelineCoordinator([
    DataCollectorAgent(name="collector"),
    DataProcessorAgent(name="processor"),
    DataAnalyzerAgent(name="analyzer")
])

print("Pipeline coordinator created")

### Pattern 2: Broadcast (Parallel)

Send same task to multiple agents in parallel.

In [ ]:
import asyncio

class BroadcastCoordinator(Agent):
    """Broadcast to multiple agents in parallel"""
    
    def __init__(self, worker_agents: list):
        super().__init__(
            name="broadcast_coordinator",
            description="Broadcasts tasks to multiple agents",
            a2a_enabled=True
        )
        self.workers = worker_agents
    
    async def broadcast(self, task: Dict[str, Any]) -> list:
        """Send task to all workers in parallel"""
        print(f"Broadcasting task to {len(self.workers)} agents...")
        
        # Create messages for all workers
        messages = [
            A2AMessage(
                from_agent=self.agent_id,
                to_agent=worker.agent_id,
                message_type="task",
                payload=task
            )
            for worker in self.workers
        ]
        
        # Send all messages in parallel
        tasks = [
            worker.handle_a2a_request(msg)
            for worker, msg in zip(self.workers, messages)
        ]
        
        results = await asyncio.gather(*tasks)
        
        print(f"Received {len(results)} responses")
        return results
    
    async def run(self, task: Dict[str, Any]) -> Dict[str, Any]:
        """Run task with all workers and aggregate"""
        results = await self.broadcast(task)
        
        # Aggregate results
        return {
            "status": "success",
            "worker_results": results,
            "consensus": self.find_consensus(results)
        }
    
    def find_consensus(self, results: list) -> Any:
        """Find consensus among worker results"""
        # Simple majority voting
        votes = {}
        for result in results:
            answer = result.get("answer")
            votes[answer] = votes.get(answer, 0) + 1
        
        return max(votes.items(), key=lambda x: x[1])[0]

print("Broadcast coordinator pattern defined")

### Pattern 3: Hierarchical (Tree Structure)

In [ ]:
class HierarchicalCoordinator(Agent):
    """Hierarchical multi-level agent system
    
    Structure:
           CEO Agent
              │
         ┌────┼────┐
         │    │    │
      Mgr1  Mgr2  Mgr3
         │    │    │
      ┌──┼─┐  │  ┌─┼──┐
      W1 W2 W3 W4 W5 W6
    """
    
    def __init__(self):
        super().__init__(
            name="ceo",
            description="Top-level coordinator",
            a2a_enabled=True
        )
        
        # Initialize hierarchy
        self.managers = {
            "research_mgr": ManagerAgent("research_mgr", domain="research"),
            "dev_mgr": ManagerAgent("dev_mgr", domain="development"),
            "qa_mgr": ManagerAgent("qa_mgr", domain="quality_assurance")
        }
    
    async def delegate_project(self, project: Dict[str, Any]) -> Dict[str, Any]:
        """Delegate project to appropriate managers"""
        print(f"CEO delegating project: {project['name']}")
        
        # Assign phases to managers
        phases = [
            ("research_mgr", {"phase": "research", "project": project}),
            ("dev_mgr", {"phase": "development", "project": project}),
            ("qa_mgr", {"phase": "testing", "project": project})
        ]
        
        results = []
        for mgr_name, task in phases:
            manager = self.managers[mgr_name]
            
            message = A2AMessage(
                from_agent=self.agent_id,
                to_agent=manager.agent_id,
                message_type="delegate",
                payload=task
            )
            
            result = await manager.handle_a2a_request(message)
            results.append(result)
        
        return {
            "project": project["name"],
            "status": "completed",
            "phase_results": results
        }

class ManagerAgent(Agent):
    """Mid-level manager agent"""
    
    def __init__(self, name: str, domain: str):
        super().__init__(
            name=name,
            description=f"Manager for {domain}",
            a2a_enabled=True
        )
        self.domain = domain
        self.workers = []  # Would have worker agents
    
    async def handle_a2a_request(self, message: A2AMessage):
        """Handle delegation from CEO"""
        task = message.payload
        print(f"  {self.name} handling {task['phase']} phase...")
        
        # Delegate to workers (simplified)
        # In real system, would send to worker agents
        
        return {
            "phase": task["phase"],
            "manager": self.name,
            "status": "completed",
            "output": f"{task['phase']} completed successfully"
        }

print("Hierarchical coordinator pattern defined")

## Part 5: A2A Message Types and Protocols

### Message Types

In [ ]:
from enum import Enum

class A2AMessageType(Enum):
    """Standard A2A message types"""
    REQUEST = "request"          # Request for action
    RESPONSE = "response"        # Response to request
    NOTIFY = "notify"            # Notification (no response needed)
    QUERY = "query"              # Information query
    DELEGATE = "delegate"        # Delegate task
    STATUS = "status"            # Status update
    ERROR = "error"              # Error notification

class A2AMessage:
    """Enhanced A2A message with full protocol support"""
    
    def __init__(
        self,
        from_agent: str,
        to_agent: str,
        message_type: A2AMessageType,
        payload: Dict[str, Any],
        context: Dict[str, Any] = None,
        correlation_id: str = None,
        priority: int = 0
    ):
        self.from_agent = from_agent
        self.to_agent = to_agent
        self.message_type = message_type
        self.payload = payload
        self.context = context or {}
        self.correlation_id = correlation_id or self._generate_id()
        self.priority = priority
        self.timestamp = datetime.now()
    
    def _generate_id(self) -> str:
        import uuid
        return str(uuid.uuid4())
    
    def to_dict(self) -> Dict:
        """Serialize to dictionary"""
        return {
            "from_agent": self.from_agent,
            "to_agent": self.to_agent,
            "message_type": self.message_type.value,
            "payload": self.payload,
            "context": self.context,
            "correlation_id": self.correlation_id,
            "priority": self.priority,
            "timestamp": self.timestamp.isoformat()
        }

print("A2A message protocol defined")

## Part 6: Production A2A Patterns

### Pattern: Request-Response with Timeout

In [ ]:
import asyncio
from datetime import datetime, timedelta

class ProductionA2AAgent(Agent):
    """Agent with production-ready A2A features"""
    
    async def send_with_timeout(
        self,
        target_agent: Agent,
        message: A2AMessage,
        timeout: float = 30.0
    ) -> Dict[str, Any]:
        """Send message with timeout"""
        try:
            response = await asyncio.wait_for(
                target_agent.handle_a2a_request(message),
                timeout=timeout
            )
            return response
        except asyncio.TimeoutError:
            return {
                "status": "error",
                "error": "Request timeout",
                "timeout": timeout
            }
    
    async def send_with_retry(
        self,
        target_agent: Agent,
        message: A2AMessage,
        max_retries: int = 3
    ) -> Dict[str, Any]:
        """Send message with retry logic"""
        for attempt in range(max_retries):
            try:
                response = await target_agent.handle_a2a_request(message)
                if response.get("status") == "success":
                    return response
            except Exception as e:
                if attempt == max_retries - 1:
                    return {
                        "status": "error",
                        "error": str(e),
                        "attempts": attempt + 1
                    }
                await asyncio.sleep(2 ** attempt)  # Exponential backoff
        
        return {"status": "error", "error": "Max retries exceeded"}

print("Production A2A patterns defined")

## Part 7: Complete Multi-Agent System Example

In [ ]:
class CompleteA2ASystem:
    """Complete multi-agent system with A2A"""
    
    def __init__(self):
        # Create specialist agents
        self.agents = {
            "researcher": ResearchAgent(),
            "writer": WriterAgent(),
            "reviewer": ReviewerAgent(),
            "publisher": PublisherAgent()
        }
        
        # Create coordinator
        self.coordinator = ContentCoordinator(self.agents)
    
    async def create_content(self, topic: str) -> Dict[str, Any]:
        """Full workflow: research → write → review → publish"""
        print(f"Starting content creation for: {topic}\n")
        
        result = await self.coordinator.run(topic)
        
        print("\nContent creation complete!")
        return result

class ReviewerAgent(Agent):
    """Agent that reviews content"""
    
    async def handle_a2a_request(self, message: A2AMessage):
        task = message.payload.get("task")
        content = message.payload.get("content")
        
        if task == "review":
            # Review content
            review = {
                "status": "approved",
                "score": 8.5,
                "feedback": "Well written, good structure",
                "suggestions": ["Add more examples"]
            }
            return {"status": "success", "review": review}
        
        return {"status": "error", "message": "Unknown task"}

class PublisherAgent(Agent):
    """Agent that publishes content"""
    
    async def handle_a2a_request(self, message: A2AMessage):
        task = message.payload.get("task")
        content = message.payload.get("content")
        
        if task == "publish":
            # Publish content
            published_url = f"https://example.com/articles/{hash(content)}"
            return {
                "status": "success",
                "url": published_url,
                "published_at": datetime.now().isoformat()
            }
        
        return {"status": "error", "message": "Unknown task"}

class ContentCoordinator(Agent):
    """Coordinates content creation workflow"""
    
    def __init__(self, agents: Dict[str, Agent]):
        super().__init__(name="content_coordinator", a2a_enabled=True)
        self.team = agents
    
    async def run(self, topic: str) -> Dict[str, Any]:
        """Execute full workflow"""
        
        # 1. Research
        print("[1/4] Research phase...")
        research = await self._send("researcher", {
            "task": "research",
            "topic": topic
        })
        
        # 2. Write
        print("[2/4] Writing phase...")
        article = await self._send("writer", {
            "task": "write_article",
            "data": research["results"]
        })
        
        # 3. Review
        print("[3/4] Review phase...")
        review = await self._send("reviewer", {
            "task": "review",
            "content": article["article"]
        })
        
        # 4. Publish (if approved)
        if review["review"]["status"] == "approved":
            print("[4/4] Publishing phase...")
            published = await self._send("publisher", {
                "task": "publish",
                "content": article["article"]
            })
            
            return {
                "status": "published",
                "url": published["url"],
                "review_score": review["review"]["score"]
            }
        
        return {
            "status": "rejected",
            "feedback": review["review"]["feedback"]
        }
    
    async def _send(self, agent_name: str, payload: Dict) -> Dict:
        """Helper to send A2A message"""
        agent = self.team[agent_name]
        message = A2AMessage(
            from_agent=self.agent_id,
            to_agent=agent.agent_id,
            message_type=A2AMessageType.REQUEST,
            payload=payload
        )
        return await agent.handle_a2a_request(message)

# Test complete system
async def test_complete_system():
    system = CompleteA2ASystem()
    result = await system.create_content("AI Agents")
    print(f"\nResult: {result}")

# await test_complete_system()
print("Complete A2A system example created")

## Key Takeaways

- **A2A Protocol** enables sophisticated multi-agent collaboration
- **Message Types** provide structure (request, response, notify, etc.)
- **Coordination Patterns**: Pipeline, Broadcast, Hierarchical
- **Production Features**: Timeouts, retries, error handling
- **Workflows** can be complex with multiple agents working together

## Best Practices

1. **Use correlation IDs** to track request-response pairs
2. **Implement timeouts** to prevent hanging
3. **Add retry logic** for resilience
4. **Log all A2A messages** for debugging
5. **Define clear interfaces** between agents
6. **Use message priority** for critical tasks
7. **Implement circuit breakers** for failing agents

## Next Steps

- Build production A2A systems
- Deploy multi-agent workflows
- Monitor agent communication
- Optimize collaboration patterns

---

**You now understand Agent-to-Agent communication with Google ADK!** 🎉